In [1]:
import sqlite3
import pandas as pd
#Kết nối database
conn = sqlite3.connect("CSDL.db")
cursor = conn.cursor()


In [2]:
cursor.execute("DROP TABLE IF EXISTS student")
cursor.execute("DROP TABLE IF EXISTS course")
#Tạo bảng student
cursor.execute('''
    CREATE TABLE IF NOT EXISTS student (
        student_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        class TEXT NOT NULL,
        course_id INTEGER,
        score REAL,
        FOREIGN KEY (course_id) REFERENCES course(id)
    )
''')
#Thêm dữ liệu vào bảng student 
cursor.executemany('''
    INSERT INTO student (student_id, name, class, course_id, score) VALUES (?, ?, ?, ?, ?)
    ''', [
        (1, "Nguyen Minh Hoang", "May Tinh", 12 ,6.7),
        (2, "Tran Thi Lan", "Kinh Te", 34, 9.2 ),
        (3, "Pham Van Nam", "Toan Tin", None, 7.9),
        (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),
        (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
        (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
        (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
        (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),
        (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),
        (10, "Cao Thi Hanh", "May Tinh", None, 7.0)
]) 


conn.commit()

In [3]:
#Tạo bảng course
cursor.execute('''
    CREATE TABLE IF NOT EXISTS course (
            id INTEGER PRIMARY KEY,
            course_name TEXT NOT NULL
            )
''')
#Thêm dữ liệu vào bảng course
cursor.executemany('''
    INSERT INTO course (id, course_name) VALUES (?, ?)
''',[
    (12, "Giai Tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
])

In [4]:
cursor.execute('''
    SELECT * FROM student, course
''')
df_descartes = pd.DataFrame(cursor.fetchall(), columns=["student_id", "name", "class", "course_id", "score", "course_id", "course_name"])
print(df_descartes) # Hiển thị kết quả


    student_id               name     class  course_id  score  course_id  \
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7         12   
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7         26   
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7         34   
3            2       Tran Thi Lan   Kinh Te       34.0    9.2         12   
4            2       Tran Thi Lan   Kinh Te       34.0    9.2         26   
5            2       Tran Thi Lan   Kinh Te       34.0    9.2         34   
6            3       Pham Van Nam  Toan Tin        NaN    7.9         12   
7            3       Pham Van Nam  Toan Tin        NaN    7.9         26   
8            3       Pham Van Nam  Toan Tin        NaN    7.9         34   
9            4     Le Thanh Huyen  Toan Tin       20.0    7.2         12   
10           4     Le Thanh Huyen  Toan Tin       20.0    7.2         26   
11           4     Le Thanh Huyen  Toan Tin       20.0    7.2         34   
12          

In [5]:
# b. INNER JOIN
query_inner = "SELECT * FROM student INNER JOIN course ON student.course_id = course.id"
df_inner = pd.read_sql(query_inner, conn)
print("INNER JOIN: ")
print(df_inner.head())

# c. LEFT JOIN
query_left = "SELECT * FROM student LEFT JOIN course ON student.course_id = course.id"
df_left = pd.read_sql(query_left, conn)
print("\nLEFT JOIN: ")
print(df_left.head())

# d. RIGHT JOIN 
query_right = "SELECT * FROM course LEFT JOIN student ON student.course_id = course.id"
df_right = pd.read_sql(query_right, conn)
print("\nRIGHT JOIN: ")
print(df_right.head())

# e. FULL OUTER JOIN (SQLite không hỗ trợ, có thể mô phỏng bằng UNION của LEFT và RIGHT JOIN)
query_full_outer = """
SELECT * FROM student LEFT JOIN course ON student.course_id = course.id
UNION
SELECT * FROM student RIGHT JOIN course ON student.course_id = course.id
"""
df_full_outer = pd.read_sql(query_full_outer, conn)
print("\nFULL OUTER JOIN: ")
print(df_full_outer.head())
conn.commit()
conn.close()


INNER JOIN: 
   student_id               name     class  course_id  score  id course_name
0           1  Nguyen Minh Hoang  May Tinh         12    6.7  12   Giai Tich
1           2       Tran Thi Lan   Kinh Te         34    9.2  34    Thong ke
2           7      Bui Tien Dung   Kinh Te         34    9.2  34    Thong ke

LEFT JOIN: 
   student_id               name     class  course_id  score    id course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12.0   Giai Tich
1           2       Tran Thi Lan   Kinh Te       34.0    9.2  34.0    Thong ke
2           3       Pham Van Nam  Toan Tin        NaN    7.9   NaN        None
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2   NaN        None
4           5        Vu Quoc Anh  May Tinh       24.0    8.0   NaN        None

RIGHT JOIN: 
   id course_name  student_id               name     class  course_id  score
0  12   Giai Tich         1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7
1  26     Tin hoc       

In [6]:
conn=sqlite3.connect('CSDL.db')
cursor=conn.cursor()
cursor.execute("""UPDATE Student
            SET course_id = (SELECT id FROM Course ORDER BY RANDOM() LIMIT 1)
            WHERE course_id IS NULL""")

cursor.execute("""DELETE FROM Student
WHERE course_id NOT IN (SELECT id FROM Course)
""")
cursor.execute('select * from Student')
rows = cursor.fetchall()
for row in rows:
    print(row)
# a. Tổng số sinh viên, điểm trung bình của từng lớp.
cursor.execute('''select class,count(student_id) ,avg(score) from Student group by class''' )
print("\nTổng số sv và điểm trung bình của từng lớp")
rows = cursor.fetchall()
for row in rows:
    print(row)
# b. Tổng số sinh viên, điểm trung bình của từng môn học.
cursor.execute('''SELECT course.course_name, COUNT(Student.student_id) AS total_students, ROUND(AVG(student.score), 2) AS avg_score
FROM Student
LEFT JOIN course ON Student.course_id = course.id
GROUP BY course.course_name;''')
print("\nTổng số sinh viên, điểm trung bình của từng môn học")
rows = cursor.fetchall()
for row in rows:
    print(row)
# c. Phân loại thi đua theo số điểm của từng môn học biết: 
# i. Điểm TB ≥  9.0: Xuất sắc. 
# ii. 6.0 ≤ Điểm TB ≤ 8.9: Tốt. 
# iii. Điểm TB < 6.0: Kém. 
query=('''select course.course_name,avg(score) ,
               case
                    when avg(score) >= 9.0 then "Xuất sắc"
                    when avg(score) between 6.0 and 8.9 then "Tốt" 
                    when avg(score)< 6.0 then "Kém" 
               end as xep_loai
               from Student left join course on Student.course_id=course.id
               group by course.course_name  ''' )
df = pd.read_sql(query, conn)
print("\nTổng số sinh viên và điểm trung bình theo lớp:")
print(df)
conn.commit()

(1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7)
(2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2)
(3, 'Pham Van Nam', 'Toan Tin', 34, 7.9)
(7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2)
(9, 'Duong Huu Phuc', 'Kinh Te', 34, 7.2)
(10, 'Cao Thi Hanh', 'May Tinh', 34, 7.0)

Tổng số sv và điểm trung bình của từng lớp
('Kinh Te', 3, 8.533333333333333)
('May Tinh', 2, 6.85)
('Toan Tin', 1, 7.9)

Tổng số sinh viên, điểm trung bình của từng môn học
('Giai Tich', 1, 6.7)
('Thong ke', 5, 8.1)

Tổng số sinh viên và điểm trung bình theo lớp:
  course_name  avg(score) xep_loai
0   Giai Tich         6.7      Tốt
1    Thong ke         8.1      Tốt


In [7]:
# a. Điểm số. 
# a. Xếp hạng theo điểm số
query = """
SELECT student_id, name, score,
       RANK() OVER (ORDER BY score DESC) AS rank
FROM Student;
"""
df = pd.read_sql(query, conn)
print("\nXếp hạng sinh viên theo điểm số:")
print(df)


# b. Điểm số theo lớp học. 
query = """
SELECT student_id, name, class, score,
       RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank
FROM Student;
"""
df = pd.read_sql(query, conn)
print("\nXếp hạng sinh viên theo điểm số trong từng lớp:")
print(df)


# c. Điểm số theo mã môn học. 
query = """
SELECT student_id, name, course_id, score,
       RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank
FROM Student
ORDER BY course_id;
"""
df = pd.read_sql(query, conn)
print("\nXếp hạng sinh viên theo điểm số trong từng môn học:")
print(df)


Xếp hạng sinh viên theo điểm số:
   student_id               name  score  rank
0           2       Tran Thi Lan    9.2     1
1           7      Bui Tien Dung    9.2     1
2           3       Pham Van Nam    7.9     3
3           9     Duong Huu Phuc    7.2     4
4          10       Cao Thi Hanh    7.0     5
5           1  Nguyen Minh Hoang    6.7     6

Xếp hạng sinh viên theo điểm số trong từng lớp:
   student_id               name     class  score  rank
0           2       Tran Thi Lan   Kinh Te    9.2     1
1           7      Bui Tien Dung   Kinh Te    9.2     1
2           9     Duong Huu Phuc   Kinh Te    7.2     3
3          10       Cao Thi Hanh  May Tinh    7.0     1
4           1  Nguyen Minh Hoang  May Tinh    6.7     2
5           3       Pham Van Nam  Toan Tin    7.9     1

Xếp hạng sinh viên theo điểm số trong từng môn học:
   student_id               name  course_id  score  rank
0           1  Nguyen Minh Hoang         12    6.7     1
1           2       Tran Thi Lan    

In [8]:
# cho biết top 3 sinh viện đạt thứ hạng cao nhất, top 3 sinh viên đạt thứ hạng thấp nhất theo 
# # từng trường hợp trên.
for criteria in ["score", "class", "course_id"]:
    query = f"""
    SELECT student_id, name, {criteria}, score,
           RANK() OVER (PARTITION BY {criteria} ORDER BY score DESC) AS rank
    FROM student;
    """
    df = pd.read_sql(query, conn)
    print(f"\nTop 3 sinh viên đạt điểm cao nhất theo {criteria}:")
    print(df[df['rank'] <= 3])
    
    query = f"""
    SELECT student_id, name, {criteria}, score,
           RANK() OVER (PARTITION BY {criteria} ORDER BY score ASC) AS rank
    FROM student;
    """
    df = pd.read_sql(query, conn)
    print(f"\nTop 3 sinh viên đạt điểm thấp nhất theo {criteria}:")
    print(df[df['rank'] <= 3])

conn.commit()


Top 3 sinh viên đạt điểm cao nhất theo score:
   student_id               name  score  score  rank
0           1  Nguyen Minh Hoang    6.7    6.7     1
1          10       Cao Thi Hanh    7.0    7.0     1
2           9     Duong Huu Phuc    7.2    7.2     1
3           3       Pham Van Nam    7.9    7.9     1
4           2       Tran Thi Lan    9.2    9.2     1
5           7      Bui Tien Dung    9.2    9.2     1

Top 3 sinh viên đạt điểm thấp nhất theo score:
   student_id               name  score  score  rank
0           1  Nguyen Minh Hoang    6.7    6.7     1
1          10       Cao Thi Hanh    7.0    7.0     1
2           9     Duong Huu Phuc    7.2    7.2     1
3           3       Pham Van Nam    7.9    7.9     1
4           2       Tran Thi Lan    9.2    9.2     1
5           7      Bui Tien Dung    9.2    9.2     1

Top 3 sinh viên đạt điểm cao nhất theo class:
   student_id               name     class  score  rank
0           2       Tran Thi Lan   Kinh Te    9.2     1
1   

In [9]:
# Thêm cột graduation_date
cursor.execute("""
    ALTER TABLE student
    ADD COLUMN graduation_date DATETIME;
""")
# Cập nhật graduation_date dựa trên thứ hạng
cursor.execute('''
WITH ranked_students AS (
    SELECT student_id, RANK() OVER (ORDER BY score DESC) AS rank
    FROM Student
)
UPDATE student
SET graduation_date = DATETIME('now', '+' || (
    SELECT rank FROM ranked_students WHERE ranked_students.student_id = Student.student_id
) || ' months');
''')

# Truy vấn lại để kiểm tra kết quả
query = '''
SELECT student_id, name, class, course_id, score, graduation_date
FROM Student
ORDER BY score DESC;
'''

df_updated = pd.read_sql_query(query, conn)
print(" Bảng Student sau khi thêm trường graduation_date:")
print(df_updated)
conn.commit()

 Bảng Student sau khi thêm trường graduation_date:
   student_id               name     class  course_id  score  \
0           2       Tran Thi Lan   Kinh Te         34    9.2   
1           7      Bui Tien Dung   Kinh Te         34    9.2   
2           3       Pham Van Nam  Toan Tin         34    7.9   
3           9     Duong Huu Phuc   Kinh Te         34    7.2   
4          10       Cao Thi Hanh  May Tinh         34    7.0   
5           1  Nguyen Minh Hoang  May Tinh         12    6.7   

       graduation_date  
0  2025-04-12 14:20:09  
1  2025-04-12 14:20:09  
2  2025-06-12 14:20:09  
3  2025-07-12 14:20:09  
4  2025-08-12 14:20:09  
5  2025-09-12 14:20:09  
